# Neuronske mreže: Predikcija B-Cell epitopa u antigenima korištenjem rekurzivne neuronske mreže (obrada ulaznih podataka)




Učitavanje inicijalne csv datoteke preuzete sa baze epitopa IEDB (https://www.iedb.org/) i početno filtriranje podataka...

In [30]:
import pandas as pd

# Učitavanje CSV datoteke
df = pd.read_csv('antigen_epitopes_table.csv')

# Izlučivanje potrebnih specifičnih stupaca
selected_columns = df[['Epitope - Name', 'Epitope - Starting Position', 'Epitope - Ending Position', 'Epitope - Source Molecule IRI']]

# Prikazivanje izlučenih stupaca
print(selected_columns)


          Epitope - Name  Epitope - Starting Position  \
0             AAALPGKCGV                         66.0   
1               AAEAACFK                         13.0   
2           AAEGATPEAKYD                        122.0   
3        AAHASARQQWELQGD                         16.0   
4          AALTKAITAMTQA                        251.0   
...                  ...                          ...   
5068     KLEEASKAADESERN                        112.0   
5069        LNSGNDERIDQL                        133.0   
5070     LRETEEAKAKIEDDY                         29.0   
5071  QAKYERAEKQIQEHEQEI                         68.0   
5072     QSLTRKISLLEEGIM                         86.0   

      Epitope - Ending Position  \
0                          75.0   
1                          20.0   
2                         133.0   
3                          30.0   
4                         263.0   
...                         ...   
5068                      126.0   
5069                      144.0   


Otvaranje linka za dohvat potpune izvorne proteinske sekvence u kojoj je smješten epitop...

In [31]:

import requests


# Funkcija za rješavanje cross-referencing
def rjesi_cross_referencing(protein_id):
    with open('cross.txt', 'a') as f:
        f.write(protein_id + ' ')    #pazi da kod svakog pokretanja pobrišeš cross.txt ako postoji, jer će se inače samo nadodavati na postojeći sadržaj

# Funkcija za obradu linkova
def process_link(link):
    if pd.isna(link) or link.strip() == '':
        return 'Link_ne_postoji'
    if 'ncbi' in link:
        # Pretvaranje NCBI linka u UniProt link
        protein_id = link.split('/')[-1].split('.')[0]
        protein_id = link.split('/')[-1].split('_')[0]
        modified_link = f'https://www.uniprot.org/uniprotkb/{protein_id}.fasta'
        response = requests.get(modified_link)
        if response.status_code == 200:
            # Izdvajanje samo sekvence proteina
            fasta_content = response.text
            sequence = ''.join(fasta_content.split('\n')[1:])
            return sequence
        else:
            modified_link = f'https://www.rcsb.org/fasta/entry/{protein_id}/display'
            response = requests.get(modified_link)
            # Izdvajanje samo sekvence proteina
            fasta_content = response.text
            sequence = ''.join(fasta_content.split('\n')[1:])
            if sequence == '':
                protein_id = link.split('/')[-1].split('.')[0]
                rjesi_cross_referencing(protein_id)  #tu rješavam problem cross-referencinga između različitih baza podataka pomocu cross.txt datoteke koju potpom ubacujem u uniprot id_mapper (online alat za konverziju id-eva između različitih baza podataka)
                return 'cross_referencing_error'
            return sequence
    elif 'uniprot' in link:
        parts = link.split('/')
        parts[3] = 'uniprotkb'  # Zamjena dijela URL-a
        parts[-1] = parts[-1].split('.')[0] + '.fasta'  # Promjena završnog dijela URL-a
        modified_link = '/'.join(parts)
        response = requests.get(modified_link)
        if response.status_code == 200:
            # Izdvajanje samo sekvence proteina
            fasta_content = response.text
            sequence = ''.join(fasta_content.split('\n')[1:])
            return sequence
        else:
            return 'INVALID_LINK'
    elif 'ontology' in link:
        return 'DELETE_ROW'
    else:
        return 'INVALID_LINK'

# Obrada zadnjeg stupca i dodavanje novog stupca Source molecule koji sadrži FASTA sekvencu izvorne molekule
selected_columns['Source molecule'] = selected_columns['Epitope - Source Molecule IRI'].apply(process_link)

# Uklanjanje redaka s 'ontology' linkovima
selected_columns = selected_columns[selected_columns['Source molecule'] != 'DELETE_ROW']

# Spremanje u novi CSV fajl
selected_columns.to_csv('final.csv', index=False)

print("Podaci su uspješno spremljeni u 'final.csv'.")


Podaci su uspješno spremljeni u 'final.csv'.


/tmp/ipykernel_27127/2274198170.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_columns['Source molecule'] = selected_columns['Epitope - Source Molecule IRI'].apply(process_link)


Konačno razrješavanje cross_referencing_problema... Pomoću online alata za konverziju id-eva između različitih baza podataka dohvaćam crossing_resolved.fasta datoteku sa pripadnim sekvencama onim redolijedom čiji su id-evi navedeni u cross.txt datoteci

In [32]:
# Učitavanje i obrada cross.txt datoteke
def obradi_cross_txt():
    with open('cross.txt', 'r') as file:
        podaci = file.read().split()
    
    vidjeni = []
    modificirani_podaci = []
    for zapis in podaci:
        if zapis not in vidjeni:
            vidjeni.append(zapis)
            modificirani_podaci.append(zapis)
    
    with open('cross_mod.txt', 'w') as file:
        file.write(' '.join(modificirani_podaci))

# Poziv funkcije
obradi_cross_txt()


cross.txt datoteku ubacujem u online alat (https://www.uniprot.org/id-mapping) za mapiranje oznaka među različitim bazama i od njega dobivam cross_solved.fasta datoteku.

In [34]:
# Definiranje liste za izbaci
izbaci = [ '351727625', '351727517', '356555875', '1448280894', '129353', '74271743', 'WP_078979437', '14916530', 'NP_776953', 'NP_002280', 'NP_776945', '356571775', 'NP_001041619', 'XP_025633971', '156891692', 'NP_001236840', '356504629', '356573919', '39725577', '356572399', 'NP_001148443', 'QHN95793', '121280', 'XP_003542129', 'JE0229', '571524280', 'XP_025610791', 'NP_001235827', '1146187866', '1448294226', '1201280913', '27922971', '356571947', '224569', '223649560', '730832', '1448311381', '1236589326', 'NP_001064761', 'XP_025656642', 'XP_007206160', 'NP_851372', '356555726', '169929', 'NP_001392340', '354134', '955351371', 'XP_019127831', '1220515990', 'XP_024010162', 'XP_046915420', 'XP_025614701', '356566592', '351724439', 'NP_001150723', '754920A', '1371542602', '356530316', 'XP_014630182', 'XP_003525290', 'NP_001041618', 'XP_006405913', 'NP_851351', 'XP_006286106', '1448288088', 'NP_001026172', 'QHO21431', '356564154', 'XP_009140083', '356520519', '1448274535', '356557386', 'XP_002423351', '351723539', 'NP_001271390', 'CAC86339', 'QZP44316', 'UST29548', 'UTA91547', 'WES12437' ]
# Učitavanje i izbacivanje zapisa iz cross_mod.txt
def izbaci_zapise():
    with open('cross_mod.txt', 'r') as file:
        modificirani_podaci = file.read().split()

    for item in izbaci:
        if item in modificirani_podaci:
            modificirani_podaci.remove(item)
    
    with open('cross_mod2.txt', 'w') as file:
        file.write(' '.join(modificirani_podaci))

# Poziv funkcije
izbaci_zapise()


Sada sa mogu spariti zapise iz cross.txt datoteke i cross_solved.fasta datoteke da riješim cross_referencing izmežu različitih baza.

In [36]:
import csv
import re

# Obrada final.csv datoteke
def obradi_final_csv():
    brojac_neuspjesnog_crossreferenciranja = 0
    brojac_nepostojecih_linkova = 0
    brojac_prekomplesnih_epitopa = 0
    
    with open('final.csv', 'r') as infile, open('final_mod.csv', 'w', newline='') as outfile:
        reader = csv.DictReader(infile)
        fieldnames = reader.fieldnames
        writer = csv.DictWriter(outfile, fieldnames=fieldnames)
        writer.writeheader()
        
        cross_podaci = open('cross_mod2.txt', 'r').read().split()
        fasta_podaci = open('cross_solved.fasta', 'r').read().split('>')[1:]
        fasta_sekvence = [''.join(f.split('\n')[1:]) for f in fasta_podaci]
        
        for row in reader:
            if 'cross_referencing_error' in row['Source molecule']:
                oznaka = re.search(r'protein/([^\.]+)', row['Epitope - Source Molecule IRI']).group(1)
                if oznaka in cross_podaci:
                    try:
                        row_index = cross_podaci.index(oznaka)
                        fasta_zapis = fasta_sekvence[row_index]
                        row['Source molecule'] = fasta_zapis
                    except IndexError as e:
                        print(f'IndexError: {e} za oznaku {oznaka} s indeksom {row_index}')
                        brojac_neuspjesnog_crossreferenciranja += 1
                        continue
                else:
                    brojac_neuspjesnog_crossreferenciranja += 1
                    continue
            
            if 'Link_ne_postoji' in row['Source molecule']:
                brojac_nepostojecih_linkova += 1
                continue
            
            if '+' in row['Epitope - Name']:
                brojac_prekomplesnih_epitopa += 1
                continue
            
            writer.writerow(row)
    
    print(f'Broj neuspjesnih cross-referenciranja: {brojac_neuspjesnog_crossreferenciranja}')
    print(f'Broj nepostojećih linkova: {brojac_nepostojecih_linkova}')
    print(f'Broj prekomplesnih epitopa: {brojac_prekomplesnih_epitopa}')

# Poziv funkcije
obradi_final_csv()


Broj neuspjesnih cross-referenciranja: 424
Broj nepostojećih linkova: 54
Broj prekomplesnih epitopa: 33


Neke dodatne provjere i statistike:

In [37]:
import csv

def ispisi_redak(filepath, redak_broj):
    with open(filepath, 'r') as file:
        reader = csv.reader(file)
        for i, row in enumerate(reader):
            if i == redak_broj:
                print(row)
                return

# Poziv funkcije za ispis 3950. retka (ako je prvi redak onaj sa naslovima stupaca, onda je 3950. redak index 3949)
ispisi_redak('final_mod.csv', 3949)


['MVDKCCKQSDINTCFGE', '582.0', '598.0', 'http://www.ncbi.nlm.nih.gov/protein/P19121.2', 'MKWVTLISFIFLFSSATSRNLQRFARDAEH']


In [42]:
import csv

def provjeri_podniz(filepath, output_filepath):
    brojac_krivo_fetchani = 0
    brojac_neodgovaraju_pozicije = 0
    brojac_manja_od_20 = 0
    brojac_manja_od_18 = 0
    brojac_manja_od_16 = 0

    valid_rows = []

    with open(filepath, 'r') as file:
        reader = csv.DictReader(file)
        for i, row in enumerate(reader, start=1):  # start=1 jer prvi redak s naslovima ne broji se
            epitope_name = row['Epitope - Name']
            source_molecule = row['Source molecule']
            start_pos = int(float(row['Epitope - Starting Position'])) - 1  # Pretvaranje u 0-indeksirani sustav
            end_pos = int(float(row['Epitope - Ending Position']))

            # Provjera podniza unutar molekule
            if epitope_name not in source_molecule:
                print(f'Redak {i + 1} ne zadovoljava uvjet. (Redni broj u izvorniku: {i})')
                brojac_krivo_fetchani += 1
                continue
            
            # Provjera da li pozicije odgovaraju imenu epitopa
            if source_molecule[start_pos:end_pos] != epitope_name:
                print(f'Redak {i + 1} ima neodgovarajuće pozicije. (Redni broj u izvorniku: {i})')
                brojac_neodgovaraju_pozicije += 1
                continue

            # Brojanje duljine epitopa
            epitope_length = len(epitope_name)
            if epitope_length < 20:
                brojac_manja_od_20 += 1
            if epitope_length < 18:
                brojac_manja_od_18 += 1
            if epitope_length < 16:
                brojac_manja_od_16 += 1

            # Dodajemo redak u listu validnih redaka
            valid_rows.append(row)

    # Pisanje validnih redaka u novi CSV fajl
    with open(output_filepath, 'w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=reader.fieldnames)
        writer.writeheader()
        writer.writerows(valid_rows)

    print(f'Brojac krivo fetchani: {brojac_krivo_fetchani}')
    print(f'Brojac neodgovarajućih pozicija: {brojac_neodgovaraju_pozicije}')
    print(f'Broj redaka gdje je duljina epitopa manja od 20: {brojac_manja_od_20}')
    print(f'Broj redaka gdje je duljina epitopa manja od 18: {brojac_manja_od_18}')
    print(f'Broj redaka gdje je duljina epitopa manja od 16: {brojac_manja_od_16}')

# Poziv funkcije za provjeru i pisanje validnih redaka u novi fajl
provjeri_podniz('final_mod.csv', 'valid_rows.csv')


Redak 9 ima neodgovarajuće pozicije. (Redni broj u izvorniku: 8)
Redak 12 ima neodgovarajuće pozicije. (Redni broj u izvorniku: 11)
Redak 18 ima neodgovarajuće pozicije. (Redni broj u izvorniku: 17)
Redak 35 ima neodgovarajuće pozicije. (Redni broj u izvorniku: 34)
Redak 41 ima neodgovarajuće pozicije. (Redni broj u izvorniku: 40)
Redak 44 ima neodgovarajuće pozicije. (Redni broj u izvorniku: 43)
Redak 57 ne zadovoljava uvjet. (Redni broj u izvorniku: 56)
Redak 69 ima neodgovarajuće pozicije. (Redni broj u izvorniku: 68)
Redak 77 ima neodgovarajuće pozicije. (Redni broj u izvorniku: 76)
Redak 84 ne zadovoljava uvjet. (Redni broj u izvorniku: 83)
Redak 93 ima neodgovarajuće pozicije. (Redni broj u izvorniku: 92)
Redak 95 ima neodgovarajuće pozicije. (Redni broj u izvorniku: 94)
Redak 96 ima neodgovarajuće pozicije. (Redni broj u izvorniku: 95)
Redak 101 ima neodgovarajuće pozicije. (Redni broj u izvorniku: 100)
Redak 102 ne zadovoljava uvjet. (Redni broj u izvorniku: 101)
Redak 104 ne z

Negativni nizovi dobiveni random izvlačenjem sekvencvi iz uniprot baze podataka. Oznaka "NE" kao nemaju epitopa, tj. neće izazvati imunološku reakciju.

In [52]:
# awk 'BEGIN {seq_count = 0} /^>/ {seq_count++} seq_count > 10000 && /^>/ {if(seqlen && length(seqlen) >= 40 && length(seqlen) <= 800) print seqlen; seqlen=""; next} seq_count > 8000 && !/^>/ {seqlen=seqlen $0} END {if (seqlen && length(seqlen) >= 20 && l
# ength(seqlen) <= 300) print seqlen}' uniprot_sprot.fasta | sort | uniq | awk 'BEGIN {print "Source molecule"} {print}' | awk 'NR <= 4098' > negative_rows.csv

# awk 'NR > 1 {total += length($0); count++} END {print "Average sequence length:", total/count}' negative_rows.csv
# Average sequence length: 181,07

#izvršavanje naredbi u terminalu za filtriranje negativnih redaka iz uniprot_sprot.fasta datoteke i spremanje u negative_rows.csv, te izračun prosječne duljine sekvenci u negative_rows.csv datoteci


Provjera podudaranja između negative_rows.csv i valid_rows.csv datoteke...

In [2]:
import csv

# Pročitaj valid_rows.csv i izvuci "Source molecule" u valid_sequences.txt
with open('valid_rows.csv', mode='r') as infile:
    reader = csv.DictReader(infile)
    source_molecules = [row['Source molecule'] for row in reader]

# Pročitaj negative_rows.csv
with open('negative_rows.csv', mode='r') as infile:
    negative_rows = [line.strip() for line in infile]

# Pronađi i ispiši podudaranje
matching_sequences = [(index + 1, seq) for index, seq in enumerate(negative_rows) if seq in source_molecules]

# Ispis rezultata
for line_num, seq in matching_sequences:
    print(f'{line_num}: {seq}')
